# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [ ]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [ ]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  df_a = -3*a**2 - 0.5*a**(-0.5)
  df_b = 3*cos(3*b) + 2.5*b**(1.5)
  df_c = c**(-2)
  return [df_a, df_b, df_c] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [ ]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
# solution - concept of limits
h=0.000001
df_a = (f(2+h,3,4) - f(2,3,4))/h
df_b = (f(2,3+h,4) - f(2,3,4))/h
df_c = (f(2,3,4+h) - f(2,3,4))/h
numerical_grad = [df_a, df_b, df_c]  # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [ ]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h=0.00001
df_a = (f(2+h,3,4) - f(2-h,3,4))/(2*h)
df_b = (f(2,3+h,4) - f(2,3-h,4))/(2*h)
df_c = (f(2,3,4+h) - f(2,3,4-h))/(2*h)
numerical_grad2 = [df_a, df_b, df_c] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390820336
OK for dim 1: expected 10.25699027111255, yours returns 10.256990271617639
OK for dim 2: expected 0.0625, yours returns 0.06250000001983835


## section 2: support for softmax

In [4]:
class Value:

    def __init__(self, data, inputs =(), oper="",label=""):
        self.label = label
        self.data = data
        self.grad = 0.0
        # backward - updates the gradients of the inputs( objects used to create) of the current object
        self.backward = lambda:None
        # inputs - contains the inputs(or objects) which are use to create the current object
        self.inputs = set(inputs)
        # oper - mathematical operation used to create the current object
        self.oper = oper
        """ inputs(or objects) which are used to create the new
            object are being stored in order to enable
            'Backwardpass' """

    # string representation of a class object/instance
    def __repr__(self):
        return f"(Value={self.data})"

    # self+other = self.__add__(other)
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        new_object = Value(data=(self.data + other.data),
                           inputs= (self, other),
                           oper="+")
        def backward():
            self.grad += (1.0)*(new_object.grad)
            other.grad += (1.0)*(new_object.grad)

        new_object.backward = backward
        return new_object

    # invoked when __add__() doesn't works that is object is second operand (eg- 2+a)
    def __radd__(self,other):
        return self+other

    # self-other = self.__sub__(other)
    def __sub__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        new_object = Value(data=(self.data - other.data),
                           inputs= (self, other),
                           oper="-")

        def backward():
            self.grad += (1.0)*(new_object.grad)
            other.grad += (-1.0)*(new_object.grad)

        new_object.backward = backward
        return new_object

    # invoked when __sub__() doesn't works that is object is second operand (eg- 2-a)
    def __rsub__(self,other):
        return (-self)+other

    # self*other = self.__mul__(other)
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        new_object = Value(data=(self.data * other.data),
                           inputs= (self, other),
                           oper="*")
        def backward():
            self.grad += (other.data)*(new_object.grad)
            other.grad += (self.data)*(new_object.grad)

        new_object.backward = backward
        return new_object

    # invoked when __add__() doesn't works that is object is second operand (eg- 2*a)
    def __rmul__(self,other):
        return self*other

    # self/div = self.__div__(other)
    def __truediv__(self,other):
        return self*(other**-1)

    # -self = self.__neg__()
    def __neg__(self):
        return self*(-1)

    # self**other = self.__pow__(other)
    def __pow__(self,other):
        assert isinstance(other, (int, float)), "Error: only supports int/float for power"
        new_object = Value(data=(self.data**other),
                           inputs=(self,),
                           oper=f"**{other}")

        # dL/dx = (dx^p/dx) * (dL/dx^p)
        def backward():
           self.grad += (other)*(self.data**(other-1))*(new_object.grad)
        new_object.backward = backward
        return new_object


    def tanh(self):
        tan_h = (math.exp(2*self.data)-1)/(math.exp(2*self.data)+1)
        new_object = Value(data=(tan_h),
                           inputs=(self,),
                           oper="tanh")
        def backward():
           self.grad += (1-tan_h**2)*(new_object.grad)
        new_object.backward = backward
        return new_object

    def exp(self):
        new_object = Value(data=(math.exp(self.data)),
                           inputs=(self,),
                           oper="exp")
        def backward():
            self.grad += (new_object.data)*(new_object.grad)
        new_object.backward = backward
        return new_object

    def log(self):
        new_object = Value(data=(math.log(self.data)),
                           inputs=(self,),
                           oper="log")
        def backward():
            self.grad += (1.0/self.data)*(new_object.grad)
        new_object.backward = backward
        return new_object

    def backward_pass(self):
        topo = []
        visited = set()
        # topological sort as our neural network is DAG
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for input in node.inputs:
                    build_topo(input) # DFS
                topo.append(node)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node.backward()

In [8]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward_pass()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [ ]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
